In [1]:
import json
from bs4 import Tag
from tqdm import tqdm
import tibetcnr_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:
file_name_code = "tibetcnr"

In [3]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

# def read_json(path, file_name):
#     try:
#         with open(path+file_name, 'r', encoding='utf-8') as openfile:
#             Loaded_file = json.load(openfile)
#             print(f"Successfully loaded: {file_name}")
#         return Loaded_file
#     except Exception as e:
#         print(f"Error loading {file_name}: {str(e)}")
#         return None

def scrape_article(url, page_key_code):
    try:
        article_content = tibetcnr_utils.scrape_tibetcnr_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 0
        for i in range(start, Total_lenght):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_{file_name_code}_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"{file_name_code}_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [4]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in tibetcnr_ALL_link_འཛམ་གླིང་སྐར་ཁུང་།.json: 20
page key name: Page འཛམ་གླིང་སྐར་ཁུང་། 19
Page key code: Page འཛམ་གླིང་སྐར་ཁུང་། 
Total page in tibetcnr_ALL_link_དམིགས་བསལ་དོ་ཁུར།.json: 20
page key name: Page དམིགས་བསལ་དོ་ཁུར། 19
Page key code: Page དམིགས་བསལ་དོ་ཁུར། 
Total page in tibetcnr_ALL_link_འོས་སྦྱོར།.json: 20
page key name: Page འོས་སྦྱོར། 19
Page key code: Page འོས་སྦྱོར། 
Total page in tibetcnr_ALL_link_པར་མཛོད།.json: 20
page key name: Page པར་མཛོད། 19
Page key code: Page པར་མཛོད། 
Total page in tibetcnr_ALL_link_གསར་འགྱུར།.json: 20
page key name: Page གསར་འགྱུར། 19
Page key code: Page གསར་འགྱུར། 
Total page in tibetcnr_ALL_link_འཚོ་བའི་རྒྱུན་ཤེས།.json: 18
page key name: Page འཚོ་བའི་རྒྱུན་ཤེས། 17
Page key code: Page འཚོ་བའི་རྒྱུན་ཤེས། 


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]


  0%|          | 0/179 [00:00<?, ?it/s]



  0%|          | 0/200 [00:00<?, ?it/s]





  0%|          | 0/200 [00:00<?, ?it/s]




  0%|          | 1/200 [00:06<22:14,  6.71s/it]



  0%|          | 1/200 [00:06<22:20,  6.73s/it]


  1%|          | 1/179 [00:06<20:12,  6.81s/it]


  1%|          | 2/179 [00:06<08:27,  2.87s/it]





  0%|          | 1/200 [00:06<22:55,  6.91s/it]

  0%|          | 1/200 [00:07<23:04,  6.96s/it]





  1%|          | 2/200 [00:07<09:39,  2.92s/it]

  2%|▏         | 3/200 [00:07<06:11,  1.88s/it]



  2%|▏         | 3/200 [00:07<06:14,  1.90s/it]

  2%|▏         | 4/200 [00:07<03:45,  1.15s/it]


  2%|▏         | 3/179 [00:07<05:00,  1.71s/it]





  2%|▏         | 3/200 [00:07<05:40,  1.73s/it]



  2%|▏         | 4/200 [00:07<04:17,  1.32s/it]

  2%|▎         | 5/200 [00:08<02:44,  1.19it/s]




  0%|          | 1/200 [00:08<24:29,  7.38s/it]





  2%|▏         | 4/200

Redirect 1: http://www.cnrmz.com/cnrmz_404/404.html







 30%|███       | 60/200 [00:47<01:19,  1.76it/s]

 30%|███       | 60/200 [00:48<01:30,  1.55it/s]




 30%|███       | 61/200 [00:48<01:10,  1.97it/s]


 34%|███▎      | 60/179 [00:48<01:27,  1.36it/s]

Redirect 1: http://www.cnrmz.com/cnrmz_404/404.html


...

Redirect 10: http://www.cnrmz.com/cnrmz_404/404.html
Redirect 7: http://www.cnrmz.com/cnrmz_404/404.html
Redirect 8: http://www.cnrmz.com/cnrmz_404/404.html
Redirect 9: http://www.cnrmz.com/cnrmz_404/404.html








100%|█████████▉| 199/200 [04:39<00:01,  1.59s/it]

Redirect 10: http://www.cnrmz.com/cnrmz_404/404.html
Redirect 6: http://www.cnrmz.com/cnrmz_404/404.html
Redirect 7: http://www.cnrmz.com/cnrmz_404/404.html
Redirect 8: http://www.cnrmz.com/cnrmz_404/404.html
Redirect 9: http://www.cnrmz.com/cnrmz_404/404.html








100%|██████████| 6/6 [04:45<00:00, 47.54s/it]

Redirect 10: http://www.cnrmz.com/cnrmz_404/404.html
Total Failure in the པར་མཛོད། article: 144
tibetcnr_ALL_content_པར་མཛོད།.json
Successfully saved: tibetcnr_ALL_content_པར་མཛོད།.json
Processed 6 files
CPU times: user 58.9 s, sys: 2.9 s, total: 1min 1s
Wall time: 4min 45s


In [5]:
# 14:44

In [ ]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files